# Common Table Expressions (CTE)

In [65]:
import pandas as pd
from DB_Fetcher import DB_Fetcher

In [48]:
fetcher = DB_Fetcher()

In [49]:
query = f"""
    CREATE TABLE Employees (
        EmployeeID int,
        FirstName string,
        LastName string,
        ManagerID int
    )
"""
# fetcher.execute_query(
#     query
# )

In [50]:
df = pd.DataFrame({
    'EmployeeID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'FirstName': ['Harper', 'Liam', 'Evelyn', 'Mason', 'Isla', 'Noah', 'Ruby', 'Caleb', 'Avery', 'Oliver'],
    'LastName': ['Westbrook', 'Carrington', 'Radcliffe', 'Albright', 'Whitman', 'Sterling', 'Lennox', 'Winslow', 'Sinclai', 'Beckett'],
    'ManagerID': [None, 1, 1, 2, 2, 3, 4, 5, 6, 6]
})

# fetcher.insert_in_db('Employees', df)

In [51]:
query = f"""
    SELECT
        *
    FROM
        Employees
"""
fetcher.execute_query_to_df(query)

,EmployeeID,FirstName,LastName,ManagerID
0,1,Harper,Westbrook,NaN
1,2,Liam,Carrington,1.0
2,3,Evelyn,Radcliffe,1.0
3,4,Mason,Albright,2.0
4,5,Isla,Whitman,2.0
5,6,Noah,Sterling,3.0
6,7,Ruby,Lennox,4.0
7,8,Caleb,Winslow,5.0
8,9,Avery,Sinclai,6.0
9,10,Oliver,Beckett,6.0


## Recursive CTE (Company Hierarchy)

This is made of 3 phases:
* Anchor Query (base case): Initial set of rows
* Recursive Query (recursive case): References the CTE itself and runs iteratively
* Termination Condition: Stop when no more rows are returned

In [ ]:
query = """
    WITH RECURSIVE employee_hierarchy AS (
        -- Anchor query: Get top-level manager
        SELECT
            EmployeeID,
            FirstName,
            LastName,
            ManagerID,
            1 AS Level
        FROM
            Employees
        WHERE
            ManagerID IS NULL

        UNION ALL
        
        -- Recursive query: Get employees reporting to the current level
        SELECT
            e.EmployeeID,
            e.FirstName,
            e.LastName,
            e.ManagerID,
            eh.Level + 1
        FROM
            Employees e
        INNER JOIN
            employee_hierarchy eh 
        ON
            e.ManagerID = eh.EmployeeID
    )

    SELECT
        *
    FROM
        employee_hierarchy;
"""
fetcher.execute_query_to_df(query)

,EmployeeID,FirstName,LastName,ManagerID,Level
0,1,Harper,Westbrook,NaN,1
1,2,Liam,Carrington,1.0,2
2,3,Evelyn,Radcliffe,1.0,2
3,4,Mason,Albright,2.0,3
4,5,Isla,Whitman,2.0,3
5,6,Noah,Sterling,3.0,3
6,7,Ruby,Lennox,4.0,4
7,8,Caleb,Winslow,5.0,4
8,9,Avery,Sinclai,6.0,4
9,10,Oliver,Beckett,6.0,4


## Recursive CTE (Calendar)

In [61]:
query = """
    SELECT
        '2024-01-01' AS Date
"""
fetcher.execute_query_to_df(query)

,Date
0,2024-01-01


In [63]:
query = """
    WITH RECURSIVE
        dates (Date)
    AS
        (
            SELECT
                '2024-01-01' AS Date
            
            UNION ALL

            SELECT
                date(Date, '+1 day')
            FROM
                dates
            WHERE
                Date < '2025-12-31'
        )

    SELECT
        *
    FROM
        dates
"""
fetcher.execute_query_to_df(query)

,Date
0,2024-01-01
1,2024-01-02
2,2024-01-03
3,2024-01-04
4,2024-01-05
...,...
726,2025-12-27
727,2025-12-28
728,2025-12-29
729,2025-12-30
